# 손가락 꼭짓점 세주는 코드(정확도 low)

In [7]:
import cv2
import mediapipe
import time

ctime=0
ptime=0

cap=cv2.VideoCapture(0)

medhands=mediapipe.solutions.hands
hands=medhands.Hands(max_num_hands=1,min_detection_confidence=0.7)
draw=mediapipe.solutions.drawing_utils

while True:
    success, img=cap.read()
    img = cv2.flip(img,1)
    imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
    res = hands.process(imgrgb)
    
    lmlist=[]
    tipids=[4,8,12,16,20] #list of all landmarks of the tips of fingers
    
    cv2.rectangle(img,(20,350),(90,440),(0,255,204),cv2.FILLED)
    cv2.rectangle(img,(20,350),(90,440),(0,0,0),5)
    
    if res.multi_hand_landmarks:
        for handlms in res.multi_hand_landmarks:
            for id,lm in enumerate(handlms.landmark):
                
                h,w,c= img.shape
                cx,cy=int(lm.x * w) , int(lm.y * h)
                lmlist.append([id,cx,cy])
                if len(lmlist) != 0 and len(lmlist)==21:
                    fingerlist=[]
                    
                    #thumb and dealing with flipping of hands
                    if lmlist[12][1] > lmlist[20][1]:
                        if lmlist[tipids[0]][1] > lmlist[tipids[0]-1][1]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)
                    else:
                        if lmlist[tipids[0]][1] < lmlist[tipids[0]-1][1]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)
                    
                    #others
                    for id in range (1,5):
                        if lmlist[tipids[id]][2] < lmlist[tipids[id]-2][2]:
                            fingerlist.append(1)
                        else:
                            fingerlist.append(0)
                    
                    
                    if len(fingerlist)!=0:
                        fingercount=fingerlist.count(1)
                    
                    
                    cv2.putText(img,str(fingercount),(25,430),cv2.FONT_HERSHEY_PLAIN,6,(0,0,0),5)
                    
                #change color of points and lines
                draw.draw_landmarks(img,handlms,medhands.HAND_CONNECTIONS,draw.DrawingSpec(color=(0,255,204),thickness=2,circle_radius=2),draw.DrawingSpec(color=(0,0,0),thickness=2,circle_radius=3))
    
    #fps counter
    ctime = time.time()
    fps=1/(ctime-ptime)
    ptime=ctime
    
    #fps display
    cv2.putText(img,f'FPS:{str(int(fps))}',(0,12),cv2.FONT_HERSHEY_PLAIN,1,(0,255,0),1)
          
    cv2.imshow("hand gestures",img)
    
    #press q to quit
    if cv2.waitKey(1) == ord('q'):
        break
    
cv2.destroyAllWindows()

# 수화로 숫자인식(제일 잘되던거)

In [6]:
import cv2
import mediapipe as mp
from google.protobuf.json_format import MessageToDict

def get_hand_labels(hand_process_results, labels_array):
    if hand_process_results.multi_handedness is not None:
        for idx, hand_handedness in enumerate(hand_process_results.multi_handedness):
            handedness_dict = MessageToDict(hand_handedness)
            class_array = handedness_dict.get('classification')
            if class_array is not None:
                for item in class_array:
                    labels_array.append(item.get('label'))
    return labels_array

def drawing_hand_landmarks(hand_process_results, right_hand_fingers, left_hand_fingers):
    if hand_process_results.multi_hand_landmarks:
        for i,hand_landmarks in enumerate(hand_process_results.multi_hand_landmarks):
            # drawing bounding box
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h

            for id, lm in enumerate(hand_landmarks.landmark):
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y

                if labels[i] == 'Right':
                    right_hand_fingers.append({'id': id, 'x': x, 'y': y})
                else:
                    left_hand_fingers.append({'id': id, 'x': x, 'y': y})
                
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(frame, labels[i], (x_min,y_min-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            # drawing hand landmarks
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    return right_hand_fingers, left_hand_fingers

def configure_fingers(fingers, right_hand_fingers, left_hand_fingers):
    if len(right_hand_fingers) > 0:
        # detect Thumbs coordinate whether inside palm or outside
        if right_hand_fingers[tipsId[0]]['x'] > right_hand_fingers[tipsId[0]-1]['x']:
            fingers.append(0)
        else:
            fingers.append(1)

        # other fingers
        for id in range(1,5):
            if right_hand_fingers[tipsId[id]]['y'] < right_hand_fingers[tipsId[id]-2]['y']:
                fingers.append(1)
            else:
                fingers.append(0)

    if len(left_hand_fingers) > 0:
        # detect Thumbs coordinate whether inside palm or outside
        if left_hand_fingers[tipsId[0]]['x'] < left_hand_fingers[tipsId[0]-1]['x']:
            fingers.append(0)
        else:
            fingers.append(1)
        # other fingers
        for id in range(1,5):
            if left_hand_fingers[tipsId[id]]['y'] < left_hand_fingers[tipsId[id]-2]['y']:
                fingers.append(1)
            else:
                fingers.append(0)
    return fingers

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.8)

tipsId = [4, 8, 12, 16, 20]

camera = cv2.VideoCapture(0)


while True:
    _, frame = camera.read()
    # flip frame
    frame =  cv2.flip(frame, 1)
    # getting frame width, height, center
    h, w, c = frame.shape
    # get hands
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    labels = []
    right_hand_fingers = []
    left_hand_fingers = []

    # Detect Hand Label (Right/Left)
    labels = get_hand_labels(results, labels)

    # Drawing Hand Landmarks
    right_hand_fingers, left_hand_fingers = drawing_hand_landmarks(results, right_hand_fingers, 
                                                                    left_hand_fingers)
    

    
    # Detect Fingers
    total_fingers = -1
    fingers = []

    fingers = configure_fingers(fingers, right_hand_fingers, left_hand_fingers)
        
    '''
    according to Malaysian Sign Language
    https://malaysiansignlanguage.weebly.com/uploads/1/0/5/8/105863499/editor/bimno_1.jpg?1496377606
    '''
    zero = [0,0,0,0,0]
    one = [0,1,0,0,0]
    two = [0,1,1,0,0]
    three = [1,1,1,0,0]
    four = [0,1,1,1,1]
    five = [1,1,1,1,1]
    six = [0,1,1,1,0]
    seven = [0,1,1,0,1]
    eight = [0,1,0,1,1]
    nine = [0,0,1,1,1]
    ten = [1,0,0,0,0]

    if fingers == zero:
        total_fingers = 0
    if fingers == one:
        total_fingers = 1
    if fingers == two:
        total_fingers = 2
    if fingers == three:
        total_fingers = 3
    if fingers == four:
        total_fingers = 4
    if fingers == five:
        total_fingers = 5
    if fingers == six:
        total_fingers = 6
    if fingers == seven:
        total_fingers = 7
    if fingers == eight:
        total_fingers = 8
    if fingers == nine:
        total_fingers = 9
    if fingers == ten:
        total_fingers = 10
    
    if total_fingers != -1:
        text = f'Number: {total_fingers}'
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, text, (100,100), font, 2, (0,0,255), 2)

    # show window frame contain live camera video
    cv2.imshow("frame", frame)


    # wait for key every 1 millisecond
    key = cv2.waitKey(1)

    # close window when click exit button
    if cv2.getWindowProperty("frame",cv2.WND_PROP_VISIBLE) == 0:        
        break

camera.release()
cv2.destroyAllWindows()

In [9]:
!pip show opencv-python

Name: opencv-python
Version: 4.6.0.66
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/skvark/opencv-python
Author: 
Author-email: 
License: MIT
Location: c:\programdata\anaconda3\lib\site-packages
Requires: numpy, numpy
Required-by: cvzone


In [10]:
!pip install opencv-python==4.5.5.64

     ---------------------------------------- 35.4/35.4 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
